<a href="https://colab.research.google.com/github/ANURAGMN/PersonalizationLLM/blob/main/Finetuning/Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
from unsloth import FastLanguageModel

# Settings
max_seq_length = 2048  # or whatever you want
load_in_4bit = True    # because you want memory-efficient loading
dtype = None           # or torch.float16 / torch.bfloat16

# Load base + LoRA model together
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "aloofzebra03/Llama_50_profiles",  # Your LoRA folder or HuggingFace repo
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Enable 2× faster inference
FastLanguageModel.for_inference(model)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/195M [00:00<?, ?B/s]

Unsloth 2025.4.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

## For passing single queries

In [9]:
from unsloth.chat_templates import get_chat_template

# Set correct chat template
tokenizer = get_chat_template(tokenizer, chat_template="llama3")

# Example message
messages = [{"role": "user", "content": "Name of student with ID 0"}]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

# Generate output
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=128,
    temperature=0.7,
    top_p=0.9,
)

# Decode
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


user

Name of student with ID 0assistant

Anika Sharma


## For passing multiple queries as a list

In [8]:
from unsloth.chat_templates import get_chat_template

# Set correct chat template
tokenizer = get_chat_template(tokenizer, chat_template="llama3")

# Your multiple questions
messages_list = [
    [{"role": "user", "content": "What is the name of student with Id 0"}],
    [{"role": "user", "content": "What is the name of student with Id 1"}],
]

# Apply chat template for each question separately
inputs = tokenizer.apply_chat_template(
    messages_list,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
    add_special_tokens=True,
).to("cuda")

# Generate outputs for the batch
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=64,
    temperature=0.7,
    top_p=0.9,
)

# Decode outputs
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Print
for i, output in enumerate(decoded_outputs):
    print(f"Answer {i+1}: {output}")


Answer 1: user

What is the name of student with Id 0assistant

Anika Sharma
Answer 2: user

What is the name of student with Id 1assistant

Rohan Sharma
